# Let's play with the moving boundary condition!

**Are we in SWAN?**

In [ ]:
%pip install --user crank-nicolson-numba

**Do we want the matplotlib interactive magicness?**

In [1]:
%matplotlib widget

**Library Import**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import scipy.integrate
from tqdm.notebook import tqdm
import crank_nicolson_numba.generic as cn
import itertools
# For parallelization
from joblib import Parallel, delayed

import nekhoroshev_tools as nt

## First, we need to define a realistic initial distribution

It's very simple and direct: if we decide to work in normalized emittance variables, and we work with a gaussian beam distribution with it's characteristic emittance, we have that
$$\rho_0(I) = e^{-I}$$
Where $I$, from now on, is expessed in sigma units.

In [3]:
def rho_0(I, damping_position=np.nan, l=np.nan):
    if np.isnan(damping_position) or np.isnan(l):
        return np.exp(-I)
    else:
        return np.exp(-I) / (1 + np.exp((I - damping_position)/l))

## Working constants

In [81]:
I_damping = 9.8
I_max = 6.0
I_star = 6.0
k = 0.33
exponent = 1/(2*k)

c = nt.standard_c(0.0, I_max, I_star, exponent)

In [82]:
cn_sampling = 50000
I_list, dI = np.linspace(0.0, I_max, cn_sampling, retstep=True)

cn_time_steps = 500
dt = nt.current_peak_time(I_damping, I_max, I_star, exponent, c)/cn_time_steps

**Values our specific analysis**

In [83]:
steps_per_sample = 20

steps_per_iteration = 25
iterations = 20
long_track_steps = steps_per_iteration * iterations

## How's the regular current for a single, constant $I_a$?

In [84]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))
data_0 = engine.get_data_with_x()

times_long, current_long = engine.current(long_track_steps*6, steps_per_sample, False)
data_long = engine.get_data_with_x()

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)
100%|██████████| 3000/3000 [01:21<00:00, 36.77it/s]


**That's how our distribution (slowly) evolves...**

(In order to see things it's necessary to work with matplotlib interactive mode and zoom a lot)

In [85]:
plt.figure()
plt.plot(data_0[0], data_0[1], label="Initial condition")
plt.plot(data_long[0], data_long[1], label="After iterations")
plt.axvline(I_max, color="black", label="$I_a$")
plt.xlabel("$I$")
plt.ylabel("$\\rho$")
plt.yscale("log")
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**And here's the current!**

In [86]:
plt.figure()
plt.plot(times_long, current_long)
plt.xlabel("$t$")
plt.ylabel("$J$")
plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## And what happens when we have $I_a$ moving outwards?

In [88]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))

t_out = []
c_out = []

for i in tqdm(range(iterations)):
    time, current = engine.current(steps_per_iteration, steps_per_sample)
    if len(t_out) > 0:
        time += t_out[-1][-1]
    t_out.append(time)
    c_out.append(current)
    engine.move_barrier_forward(0.2)

  0%|          | 0/20 [00:00<?, ?it/s]

In [89]:
plt.figure()

for i in range(iterations):
    plt.plot(t_out[i], c_out[i])
    if i != iterations - 1:
        plt.plot([t_out[i][-1], t_out[i][-1]], [c_out[i][-1], c_out[i+1][0]], c="black")

plt.plot(times_long, current_long, c="C0", linestyle="dashed")
#plt.yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Experimenting with a bit of dancing!

In [90]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))

t_out = []
c_out = []
I_max_list = []
set_list = []

t_relative_out = []
t_absolute_out = []

t_out_partial = []
c_out_partial = []
I_max_list_partial = []
set_list_partial = []

t_relative_out_partial = []
t_absolute_out_partial = []

for j in tqdm(range(6)):
    if j != 0:
        engine.move_barrier_backward(0.2 * iterations)

    for i in tqdm(range(iterations)):
        time, current = engine.current(steps_per_iteration, steps_per_sample)
        if len(t_out) > 0:
            abs_time = time + t_out[-1][-1]
        else:
            abs_time = time.copy()
        t_out.append(abs_time)
        t_absolute_out.append(abs_time)
        t_relative_out.append(time)
        c_out.append(current)
        I_max_list.append(engine.I_max)
        set_list.append(i)
        if i != 0:
            t_out_partial.append(abs_time)
            t_absolute_out_partial.append(abs_time)
            t_relative_out_partial.append(time)
            c_out_partial.append(current)
            I_max_list_partial.append(engine.I_max)
            set_list_partial.append(i)
        engine.move_barrier_forward(0.2)

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [101]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
for i in range(len(t_out)):
    ax2.plot([t_out[i][0],t_out[i][-1], t_out[i][-1]], [I_max_list[i],I_max_list[i],(I_max_list[i+1] if i+1 != len(I_max_list) else I_max_list[i])], c="black", alpha=0.5)
    ax1.plot(t_out[i], c_out[i])
    if i != len(t_out) - 1:
        ax1.plot([t_out[i][-1], t_out[i][-1]], [c_out[i][-1], c_out[i+1][0]], c="black")
        
ax1.plot(times_long, current_long, c="C0", linestyle="dashed")
ax1.set_yscale("log")
ax1.set_ylabel("Current")
ax1.set_xlabel("time")
ax2.set_ylabel("Barrier position")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Barrier position')

In [102]:
plt.figure()
plt.scatter(I_max_list_partial[1:], [np.min(c) for c in c_out_partial[1:]], label="minimum registered")
plt.scatter(I_max_list_partial[1:], [np.max(c) for c in c_out_partial[1:]], label="maximum registered")
plt.yscale("log")
plt.ylabel("Current")
plt.xlabel("Barrier position")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'Barrier position')

In [93]:
print(t_relative_out_partial[0][1], t_relative_out_partial[0][-2])
print(t_relative_out_partial[-1][1], t_relative_out_partial[-1][-2])

0.12474482303012244 2.8691309296928162
0.12474482303012244 2.8691309296928162


In [94]:
def strange_fit(x_data, I_star, exponent):
    print(I_star, exponent)
    c_1 = np.empty(len(x_data))
    c_2 = np.empty(len(x_data))
    for i, x in enumerate(x_data):
        t_1 = x[0]
        t_2 = x[1]
        I_0 = x[2]
        I_a = x[3]
        c_1[i] = nt.current_point(t_1, I_0, I_a, I_star, exponent, c)
        c_2[i] = nt.current_point(t_2, I_0, I_a, I_star, exponent, c)
    print(c_1/c_2)
    return c_1 / c_2

In [95]:
x_data[0]

(0.0010953383204203287,
 0.02519278136966756,
 10.000004000079802,
 10.200004000079801)

In [96]:
nt.current_point(x_data[0][0], x_data[0][2], x_data[0][3], I_star, exponent, c)

0.9250814553944461

In [97]:
nt.current_point(x_data[0][1], x_data[0][2], x_data[0][3], I_star, exponent, c)

9.254872895466335

In [105]:
plt.figure()
plt.plot(np.linspace(0, x_data[0][1],100), 
         [nt.current_point(x, x_data[0][3]-0.2, x_data[0][3], I_star, exponent, c) for x in np.linspace(0, x_data[0][1], 100)])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [99]:
x_data = list(zip(
    [t[1] for t in t_relative_out_partial],
    [t[-2] for t in t_relative_out_partial],
    [I-0.2 for I in I_max_list_partial],
    I_max_list_partial,
))

y_data = [c[1]/c[-2] for c in c_out_partial]

In [100]:
popt, pcov = scipy.optimize.curve_fit(
    strange_fit,
    x_data,
    y_data,
    p0=(I_star, exponent)
)

6.0 1.5151515151515151
[ 94.35608665  95.71617626  96.86528014  97.84327001  98.68155408
  99.40501768 100.03349635 100.58289625 101.06604927 101.49336851
 101.87335336 102.21298051 102.51800853 102.79321612 103.04258973
 103.26947186 103.47667904 103.66659605 103.84125161  94.36230531
  95.72141131  96.86972001  97.84706274  98.68481662  99.40784299
 100.03595875 100.58505554 101.06795383 101.49505773 101.87485947
 102.21433005 102.51922347 102.79431474 103.04358733 103.27038128
 103.47751115 103.66736006 103.84195537  94.36851937  95.72664265
  96.87415688  97.85085301  98.68807714  99.41066662 100.03841977
 100.58721367 101.06985741 101.49674611 101.87636487 102.21567899
 102.52043789 102.79541292 103.04458453 103.27129036 103.47834295
 103.66812379 103.8426589   94.37472884  95.7318703   96.87859074
  97.85464083  98.69133565  99.41348859 100.04087939 100.58937063
 101.07176001 101.49843366 101.87786956 102.21702732 102.52165179
 102.79651064 103.04558134 103.27219909 103.47917445 

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:78: RuntimeWarning: divide by zero encountered in double_scalars
  return -scipy.integrate.quad(lambda x: 1/np.sqrt(D(x, I_star, exponent, c)), I, I_max)[0]
/home/camontan/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: divide by zero encountered in func (vectorized)
  outputs = ufunc(*inputs)
/home/camontan/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2197: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)
/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:207: RuntimeWarning: invalid value encountered in double_scalars
  return -x(I, I_max, I_star, exponent, c) / (t * np.sqrt(2*np.pi*t)) * np.exp(-(x(I, I_max, I_star, exponent, c)+((nu(I, I_star, exponent, c)/2)*t))**2/(2*t))
<ipython-input-94-c

[1.09521216e+02 1.15123637e+02 1.35227032e+02 2.08833997e+02
 4.95781068e+02 1.14692256e+03 2.89937851e+02 6.86606856e-13
 0.00000000e+00            nan            nan            nan
            nan            nan            nan            nan
            nan            nan            nan 1.09532641e+02
 1.15165610e+02 1.35376705e+02 2.09398439e+02 4.97936327e+02
 1.14838110e+03 2.84705542e+02 3.70839354e-13 0.00000000e+00
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan            nan 1.09544130e+02 1.15207808e+02
 1.35527186e+02 2.09966157e+02 5.00101980e+02 1.14980256e+03
 2.79513953e+02 1.97770217e-13 0.00000000e+00            nan
            nan            nan            nan            nan
            nan            nan            nan            nan
            nan 1.09555683e+02 1.15250233e+02 1.35678481e+02
 2.10537171e+02 5.02278044e+02 1.15118653e+03 2.74363701e+02
 1.04113357e-13 0.000000

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: invalid value encountered in power
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)
/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:78: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return -scipy.integrate.quad(lambda x: 1/np.sqrt(D(x, I_star, exponent, c)), I, I_max)[0]
/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:123: RuntimeWarning: invalid value encountered in power
  return (np.sqrt(c) * exponent / I) * np.power(I_star/I, exponent) * np.exp(-np.power(I_star/I, exponent))


[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
-0.32973629688206074 -0.039378230065301
[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
0.09037654283046682 -0.227395787

KeyboardInterrupt: 

In [106]:
plt.figure()
plt.scatter(I_max_list_partial[1:], [np.min(c)/np.max(c) for c in c_out_partial[1:]])

plt.xlabel("Barrier position")
plt.ylabel("Current minimum / Current maximum")
plt.title("Attempt in current 'normalization'")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, "Attempt in current 'normalization'")

## And what happens when we have $I_a$ moving inwards?

In [16]:
engine = cn.cn_generic(0, I_max, rho_0(I_list, I_damping, dI*5), dt, lambda x: nt.D(x, I_star, exponent, c, True))

t_in = []
c_in = []

for i in tqdm(range(iterations)):
    time, current = engine.current(steps_per_iteration, steps_per_sample)
    if len(t_in) > 0:
        time += t_in[-1][-1]
    t_in.append(time)
    c_in.append(current)
    engine.move_barrier_backward(0.5)

/home/camontan/Insync/carlo.montanari3@studio.unibo.it/OneDrive_Biz/projects/new_games_with_diffusion/nekhoroshev_tools.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  return c * np.exp(-2*np.power(I_star/I, exponent)) * (0.5 if halved else 1.0)


  0%|          | 0/20 [00:00<?, ?it/s]

AssertionError: 

In [ ]:
plt.figure()
data = engine.get_data_with_x()
plt.plot(data[0], data[1])

In [ ]:
plt.figure()

for i in range(iterations):
    plt.plot(t_in[i], c_in[i])
    if i != iterations - 1:
        plt.plot([t_in[i][-1], t_in[i][-1]], [c_in[i][-1], c_in[i+1][0]], color="black")

plt.plot(times_long, current_long, c="C0", linestyle="dashed")

# A couple of attempts in fitting an exponential law

In [15]:
def basic_exponential(x, k, c, a):
    return c * np.exp(-k*(x+a)**2)/(x+a)**(3/2)

In [16]:
popts = []
pcovs = []

for i in range(iterations):
    t = t_in[i] if i == 0 else t_in[i] - t_in[i-1][-1]
    current = c_in[i]
    popt, pcov = scipy.optimize.curve_fit(
        basic_exponential,
        t,
        current
    )
    popts.append(popt)
    pcovs.append(pcov)

<ipython-input-15-1b78da6456a0>:2: RuntimeWarning: invalid value encountered in power
  return c * np.exp(-k*(x+a)**2)/(x+a)**(3/2)


In [17]:
long_popt, long_pcov = scipy.optimize.curve_fit(
    basic_exponential,
    times_long,
    current_long
)

<ipython-input-15-1b78da6456a0>:2: RuntimeWarning: invalid value encountered in power
  return c * np.exp(-k*(x+a)**2)/(x+a)**(3/2)


In [18]:
plt.figure()

for i in range(iterations):
    plt.plot(t_in[i], c_in[i])
    t = t_in[i] if i == 0 else t_in[i] - t_in[i-1][-1]
    plt.plot(
        t_in[i],
        basic_exponential(t, popts[i][0], popts[i][1], popts[i][2]),
        color="grey",
        linestyle="dashed"
    )
    if i != iterations - 1:
        plt.plot([t_in[i][-1], t_in[i][-1]], [c_in[i][-1], c_in[i+1][0]], color="black")

plt.plot(times_long, current_long, c="C0", linestyle="dashed")
plt.plot(
    times_long,
    basic_exponential(
        times_long,
        long_popt[0],
        long_popt[1],
        long_popt[2]
    ),
    color="black",
    linestyle="dashed"
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## How's the known and observed behaviour in experimental literature?

### From LHC beam halo dynamics paper:
![scheme](literature_1.png)

### From Stancari's paper:
![scheme2](literature_2.png)

### Quick basic remarks

* We can observe good similarities with the behaviour in our model and the established experimental data:
    1. When we have an outward step, the current has a big dip and then a fast recover to the original current;
    2. When we have an inward step, the current has a big peak (N.B. here we are neglecting the beam part cutted away in the process, but it can be considered and included in the plot), and then a fast relaxation to the original current;
* There is just an element that extremely differs from the experimental data, the big change in module in the signal depending on the jaw position. Can this be related to the different closure of the jaws at the different positions?